In [ ]:
import sys
sys.path.append('..')

import sciluigi as sl

from src import Receipt, load_data
from src.steps import download_data, segment, get_task

In [ ]:
receipt_path = r'C:\Users\formanj\GitHub\AngelFISH\AngelFISH\examples\new_pipeline.json'
receipt = Receipt(path=receipt_path)

In [ ]:
class AngelFISHLuigiTask(sl.ExternalTask):
    receipt_path = sl.Parameter()
    step_name = sl.Parameter()

    def out_doneflag(self):
        return sl.TargetInfo(self, f'tmp/flags/{self.step_name}.done')

    def run(self):
        receipt = Receipt(path=self.receipt_path)
        task_class = get_task(receipt['steps'][self.step_name]['task_name'])
        task = task_class(receipt, self.step_name)
        task.process()

        # Create done flag to satisfy Luigi
        with open(self.out_doneflag().path, 'w') as f:
            f.write('done')

In [ ]:
class AngelFISHWorkflow(sl.WorkflowTask):
    receipt_path = sl.Parameter()

    def workflow(self):
        receipt = Receipt(path=self.receipt_path)
        step_order = receipt['step_order']

        previous_task = None
        task_refs = []

        for step_name in step_order:
            step_task = sl.new_task(
                step_name,
                AngelFISHLuigiTask,
                self,
                receipt_path=self.receipt_path,
                step_name=step_name
            )

            if previous_task:
                # create dependency chain
                step_task.in_upstream = previous_task.out_doneflag

            previous_task = step_task
            task_refs.append(step_task)

        # Return the final task so Luigi knows when the workflow is "done"
        return task_refs[-1]